In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-136841
Azure region: southcentralus
Subscription id: 6971f5ac-8af1-446e-8034-05acea24681f
Resource group: aml-quickstarts-136841


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "computeluster"
try:
    compute_cluster = ComputeTarget(ws, cluster_name)
    print("Found exisitng compute cluster")
except ComputeTargetException:
    print("Creating new computer cluster")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)

    compute_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    print("Created computer cluster")

Found exisitng compute cluster


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': choice(0.1, 0.5, 1.0, 5.0),
    '--max_iter': choice(10, 20, 30)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=3, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='.', entry_script='train.py', compute_target=compute_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                    hyperparameter_sampling=ps,
                                    primary_metric_name="Accuracy",
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=12,
                                    max_concurrent_runs=4,
                                    policy=policy)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_ff790c4e-e9e9-4293-985b-d45e962c0a12
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_ff790c4e-e9e9-4293-985b-d45e962c0a12?wsid=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourcegroups/aml-quickstarts-136841/workspaces/quick-starts-ws-136841

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-02-01T15:51:19.391510][API][INFO]Experiment created<END>\n""<START>[2021-02-01T15:51:20.294502][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2021-02-01T15:51:20.6044939Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-02-01T15:51:20.452312][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"


In [5]:
print(hyperdrive_run.get_best_run_by_primary_metric())

Run(Experiment: udacity-project,
Id: HD_ff790c4e-e9e9-4293-985b-d45e962c0a12_11,
Type: azureml.scriptrun,
Status: Completed)


In [6]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print(f"Best Run id: {best_run.id}")
print(f"Accuracy: {best_run_metrics['Accuracy']}")
print(f"Regularization Rate: {parameter_values[1]}")
print(f"Number of Iterations: {parameter_values[3]}")

Best Run id: HD_ff790c4e-e9e9-4293-985b-d45e962c0a12_11
Accuracy: 0.9147851420247632
Regularization Rate: 1
Number of Iterations: 30


In [7]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_ff790c4e-e9e9-4293-985b-d45e962c0a12_11,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

data_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path=data_path)

In [9]:
import pandas as pd

from train import clean_data


# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    compute_target='computeluster',
    n_cross_validations=5)

In [12]:
# Submit your automl run
automl_exp = Experiment(ws, 'automl')
automl_run = exp.submit(automl_config, show_output=True)

RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Running on remote.
No run_configuration provided, running on computeluster with default configuration
Running on remote compute: computeluster
Parent Run ID: AutoML_c1177687-81ea-48c6-b57c-591cc091cf4b

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Generating individually featurized CV splits.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can

In [13]:
# Retrieve and save your best automl model.

best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

best_run.download_file('/outputs/model.pkl', 'automl_best_model.pkl')

Run(Experiment: udacity-project,
Id: AutoML_c1177687-81ea-48c6-b57c-591cc091cf4b_21,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_leaf=0.01,
                                                                                                    min_samples_split=0.2442105263157895,
  

In [14]:
compute_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

